* 中文词向量下载地址：https://github.com/Embedding/Chinese-Word-Vectors
* 数据集：https://pan.baidu.com/s/1oObY4A_Ovo1CY00UrgbBKg 提取码：kth7 

In [1]:
%matplotlib inline 
# 魔法命令，使用后画图不用show了

import numpy as np
import matplotlib.pyplot as plt
import re# 引入正则

import warnings
warnings.filterwarnings("ignore")

# 1.解压词向量并加载

## 1.1解压词向量

In [2]:
import bz2# 用来解压文件

In [4]:
with open("./data/embeddings/sgns.zhihu.bigram", 'wb') as new_file, open("./data/embeddings/sgns.zhihu.bigram.bz2", 'rb') as file:
    decompressor = bz2.BZ2Decompressor()
    for data in iter(lambda : file.read(100 * 1024), b''):
        new_file.write(decompressor.decompress(data))

## 1.2加载词向量

In [2]:
from gensim.models import KeyedVectors# gensim用来加载预训练词向量

In [3]:
cn_model = KeyedVectors.load_word2vec_format('./data/embeddings/sgns.zhihu.bigram', 
                                             binary=False,
                                             unicode_errors="ignore")

# 2.语料预处理

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 2.1读取原始文本

In [5]:
import os

pos_path='./data/4000/pos'
neg_path='./data/4000/neg'

pos_txts = os.listdir(pos_path)#pos文件夹下所有文件的名称
neg_txts = os.listdir(neg_path)#neg文件夹下所有文件的名称

train_texts_orig = []
train_target = []

for pos_txt in pos_txts:
    if not os.path.isdir(pos_txt):#判断是否是文件夹
        with open(pos_path+'/'+pos_txt, "r", encoding="utf-8") as f:
            lines=f.readlines()
            train_texts_orig.append(''.join(lines))
            train_target.append(1)
            
for neg_txt in neg_txts:
    if not os.path.isdir(neg_txt):#判断是否是文件夹
        with open(neg_path+'/'+neg_txt, "r", encoding="utf-8") as f:
            lines=f.readlines()
            train_texts_orig.append(''.join(lines))
            train_target.append(0)

In [6]:
print(train_texts_orig[2])

早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。

房间本身很好。






In [7]:
re_text=re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",train_texts_orig[2])
print(re_text)

早餐太差无论去多少人那边也不加食品的酒店应该重视一下这个问题了房间本身很好


In [8]:
print(train_target[2])

1


## 2.2进行分词和tokenize

1. 去掉每个样本的标点符号；
2. 用jieba分词，得到存放分词结果的list—cut_list
3. 将分词结果cut_list索引化，这样每一例评价的文本变成一段索引数字，对应着预训练词向量模型中的词。

* train_tokens：list of list，list中含有4000个小list，对应每一条评价分词索引。

In [10]:
import jieba # 结巴分词

In [11]:
train_tokens = []
for text in train_texts_orig:
    # 去掉标点
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    # 结巴分词
    cut_list = jieba.lcut(text)

    for i, word in enumerate(cut_list): # enumerate()
        try:
            # 将词转换为索引index
            cut_list[i] = cn_model.vocab[word].index
        except KeyError:
            # 如果词不在字典中，则输出0
            cut_list[i] = 0
    train_tokens.append(cut_list)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
Loading model cost 4.725 seconds.
Prefix dict has been built successfully.


## 2.3索引长度标准化

因为每段评语的长度是不一样的，如果单纯取最长的一个评语，并把其他评填充成同样的长度，这样十分浪费计算资源，所以取一个折衷的长度。

In [12]:
# 获得所有tokens的长度
num_tokens = [len(tokens) for tokens in train_tokens]
num_tokens = np.array(num_tokens)
# 取tokens平均值并加上两个tokens的标准差，
# 假设tokens长度的分布为正态分布，则max_tokens这个值可以涵盖95%左右的样本
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

236

### padding（填充）和truncating（修剪）

我们把文本转换为tokens（索引）之后，每一串索引的长度并不相等，所以为了方便模型的训练我们需要把索引的长度标准化，上面我们选择了236这个可以涵盖95%训练样本的长度，接下来我们进行padding和truncating，我们一般采用'pre'的方法，这会在文本索引的前面填充0，因为根据一些研究资料中的实践，如果在文本索引后面填充0的话，会对模型造成一些不良影响。 

进行padding和truncating， 输入的train_tokens是一个list
返回的train_pad是一个numpy array

In [13]:
train_pad = pad_sequences(train_tokens, maxlen=max_tokens,
                            padding='pre', truncating='pre')

In [27]:
print(type(train_pad))

<class 'numpy.ndarray'>


## 2.4准备Embedding Matrix

现在来为模型准备embedding matrix（词向量矩阵），根据keras的要求，需要准备一个维度为 (𝑛𝑢𝑚𝑤𝑜𝑟𝑑𝑠,𝑒𝑚𝑏𝑒𝑑𝑑𝑖𝑛𝑔𝑑𝑖𝑚) 的矩阵，num words代表使用的词汇的数量，emdedding dimension在现在使用的预训练词向量模型中是300，每一个词汇都用一个长度为300的向量表示。
注意只选择使用前50k个使用频率最高的词，在这个预训练词向量模型中，一共有260万词汇量，如果全部使用在分类问题上会很浪费计算资源，因为训练样本很小，一共只有4k，如果有100k，200k甚至更多的训练样本时，在分类问题上可以考虑减少使用的词汇量。

In [14]:
num_words = 50000
embedding_dim=300
# 初始化embedding_matrix，之后在keras上进行应用
embedding_matrix = np.zeros((num_words, embedding_dim))
# embedding_matrix为一个 [num_words，embedding_dim] 的矩阵
# 维度为 50000 * 300
for i in range(num_words):
    embedding_matrix[i,:] = cn_model[cn_model.index2word[i]]
embedding_matrix = embedding_matrix.astype('float32')
# 检查index是否对应，
# 输出300意义为长度为300的embedding向量一一对应
np.sum( cn_model[cn_model.index2word[333]] == embedding_matrix[333] )

300

In [15]:
# 超出五万个词向量的词用0代替
train_pad[ train_pad>=num_words ] = 0

# 准备target向量，前2000样本为1，后2000为0
train_target = np.array(train_target)

# 3.训练语料

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding, LSTM, Bidirectional
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

## 3.1划分训练集和测试集

In [16]:
# 进行训练和测试样本的分割
from sklearn.model_selection import train_test_split
# 90%的样本用来训练，剩余10%用来测试
X_train, X_test, y_train, y_test = train_test_split(train_pad,
                                                    train_target,
                                                    test_size=0.1,
                                                    random_state=12)

## 3.2搭建网络结构

用keras搭建LSTM模型，模型的第一层是Embedding层，只有当我们把tokens索引转换为词向量矩阵之后，才可以用神经网络对文本进行处理。 keras提供了Embedding接口，避免了繁琐的稀疏矩阵操作。在Embedding层我们输入的矩阵为：(𝑏𝑎𝑡𝑐ℎ𝑠𝑖𝑧𝑒,𝑚𝑎𝑥𝑡𝑜𝑘𝑒𝑛𝑠)输出矩阵为:(𝑏𝑎𝑡𝑐ℎ𝑠𝑖𝑧𝑒,𝑚𝑎𝑥𝑡𝑜𝑘𝑒𝑛𝑠,𝑒𝑚𝑏𝑒𝑑𝑑𝑖𝑛𝑔𝑑𝑖𝑚)

In [17]:
model = Sequential()
model.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_tokens,
                    trainable=False))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(LSTM(units=16, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

## 3.3模型配置

### 模型保存（断点续训）、early stoping、学习率

In [18]:
# 建立一个权重的存储点
path_checkpoint = './checkpoint/sentiment_checkpoint.keras'
checkpoint = ModelCheckpoint(filepath=path_checkpoint, monitor='val_loss',
                                      verbose=1, save_weights_only=True,
                                      save_best_only=True)

In [ ]:
# 尝试加载已训练模型
try:
    model.load_weights(path_checkpoint)
except Exception as e:
    print(e)

In [19]:
# 定义early stoping如果3个epoch内validation loss没有改善则停止训练
earlystopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

# 自动降低learning rate
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1, min_lr=1e-8, patience=0,
                                       verbose=1)
# 定义callback函数
callbacks = [
    earlystopping, 
#    checkpoint,
    lr_reduction
]

In [21]:
model.compile(optimizer=Adam(lr=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 3.4训练模型

In [22]:
# 开始训练
model.fit(X_train, y_train,validation_split=0.1,epochs=20,batch_size=128)#,callbacks=callbacks)

Train on 3240 samples, validate on 360 samples
Epoch 1/20
3240/3240 [==============================] - 137s 42ms/sample - loss: 0.5870 - accuracy: 0.7019 - val_loss: 0.5294 - val_accuracy: 0.7444
Epoch 2/20
3240/3240 [==============================] - 115s 36ms/sample - loss: 0.4264 - accuracy: 0.8154 - val_loss: 0.3796 - val_accuracy: 0.8417
Epoch 3/20
3240/3240 [==============================] - 103s 32ms/sample - loss: 0.3456 - accuracy: 0.8670 - val_loss: 0.3539 - val_accuracy: 0.8694
Epoch 4/20
3240/3240 [==============================] - 136s 42ms/sample - loss: 0.2955 - accuracy: 0.8846 - val_loss: 0.3215 - val_accuracy: 0.8722
Epoch 5/20
3240/3240 [==============================] - 137s 42ms/sample - loss: 0.2675 - accuracy: 0.9009 - val_loss: 0.2888 - val_accuracy: 0.8917
Epoch 6/20
3240/3240 [==============================] - 130s 40ms/sample - loss: 0.2429 - accuracy: 0.9108 - val_loss: 0.3560 - val_accuracy: 0.8556
Epoch 7/20
3240/3240 [==============================] - 123

## 3.5应用于测试集
首先对测试样本进行预测，得到了还算满意的准确度。之后我们定义一个预测函数，来预测输入的文本的极性，可见模型对于否定句和一些简单的逻辑结构都可以进行准确的判断。

In [23]:
result = model.evaluate(X_test, y_test)
print('Accuracy:{0:.2%}'.format(result[1]))

400/400 [==============================] - 6s 15ms/sample - loss: 0.3737 - accuracy: 0.8825
Accuracy:88.25%


# 4.预测

In [24]:
def predict_sentiment(text):
    print(text)
    # 去标点
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    # 分词
    cut = jieba.cut(text)
    cut_list = [ i for i in cut ]
    # tokenize
    for i, word in enumerate(cut_list):
        try:
            cut_list[i] = cn_model.vocab[word].index
            if cut_list[i] >= 50000:
                cut_list[i] = 0
        except KeyError:
            cut_list[i] = 0
    # padding
    tokens_pad = pad_sequences([cut_list], maxlen=max_tokens,
                           padding='pre', truncating='pre')
    # 预测
    result = model.predict(x=tokens_pad)
    coef = result[0][0]
    if coef >= 0.5:
        print('是一例正面评价','output=%.2f'%coef)
    else:
        print('是一例负面评价','output=%.2f'%coef)

In [25]:
test_list = [
    '酒店设施不是新的，服务态度很不好',
    '酒店卫生条件非常不好',
    '床铺非常舒适',
    '房间很凉，不给开暖气',
    '房间很凉爽，空调冷气很足',
    '酒店环境不好，住宿体验很不好',
    '房间隔音不到位' ,
    '晚上回来发现没有打扫卫生',
    '因为过节所以要我临时加钱，比团购的价格贵'
]
for text in test_list:
    predict_sentiment(text)

酒店设施不是新的，服务态度很不好
是一例负面评价 output=0.05
酒店卫生条件非常不好
是一例负面评价 output=0.03
床铺非常舒适
是一例正面评价 output=0.92
房间很凉，不给开暖气
是一例负面评价 output=0.28
房间很凉爽，空调冷气很足
是一例正面评价 output=0.66
酒店环境不好，住宿体验很不好
是一例负面评价 output=0.02
房间隔音不到位
是一例负面评价 output=0.25
晚上回来发现没有打扫卫生
是一例负面评价 output=0.09
因为过节所以要我临时加钱，比团购的价格贵
是一例负面评价 output=0.02
